In [11]:
# 최신순으로 최근 50개 리뷰 크롤링
# 날짜, 리뷰 내용
# 크롬 108.xx 버전

In [ ]:
#!pip install selenium

In [81]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time

In [82]:
cafeName = 'Auspice coffee'
cafeNum = '0507-1305-0628'
cafeID = ""

In [83]:
def findCafe(phone, name):
    url = f'https://m.map.naver.com/search2/searchMore.naver?query={phone}&page=1'
    response = requests.get(url)
    places = response.json()['result']['site']['list']
    for place in places:
        if place['name'] == name:
            return place['id'][1:]

In [84]:
cafeID = findCafe(cafeNum, cafeName)
cafeID

'1143065220'

In [85]:
finalurl = f'https://m.place.naver.com/restaurant/{cafeID}/review/visitor?reviewSort=recent'

In [86]:
driver = webdriver.Chrome()
driver.get(finalurl)

In [87]:
i=0
while i<5: 
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP) 
            time.sleep(3) 
            
            driver.find_element(By.CSS_SELECTOR, '.lfH3O > a').click() 
            time.sleep(3)
            i+=1
            
        except NoSuchElementException: 
            print('-더보기 버튼 모두 클릭 완료-') 
            break 

In [9]:
#!pip install lxml

In [88]:
html = driver.page_source 
soup = BeautifulSoup(html, 'lxml')

In [89]:
one_review = soup.select('.YeINN')
review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
print('리뷰 총 개수 : '+str(review_num))

리뷰 총 개수 : 60


In [93]:
selector = '.eCPGL > .YeINN > div.ZZ4OK.IwhtZ'
# > .xHaT3 > .zPfVt
elements = driver.find_elements(By.CSS_SELECTOR, selector)

data = []

for element in elements:
    data.append({"review" : element.text})

df = pd.DataFrame(data)
df


,review
0,가족들과 포항 여행 간 김에 오션뷰로 되어진 카페를 검색하니 어스피스가 나와서 가게...
1,맛잇아요
2,어스
3,뷰 맛집에 커피 맛집
4,뷰겁나좋아요!!! 주차장도 넓고 인테리어 멋져서 사진찍을맛 나요 ㅎㅎ 바다보며 물멍...
5,구룡포에서 호미곶 가는 길에 만난 뷰 맛집
6,건물이이뻐요
7,비싼편이지만 뷰값으로ㅎ
8,커피가 맛있고 뷰가 좋아요😊
9,좋아요


In [99]:
df.to_csv(f"{cafeName}.csv", encoding='utf-8-sig')

In [1]:
print('hello')

hello
